In [13]:
from trade.binanceExch import Client
from trade.binanceExch.indicators.MA import MA
from dotenv import load_dotenv
import pandas as pd


load_dotenv('/home/jb/PycharmProjects/binanceTradeDj/binanceTrade/.env')

# client = Client(base_url="https://testnet.binance.vision")
client = Client(test_mode=True)

In [14]:
columns = ['Open time','Open','High','Low','Close','Volume','Close time','Quote asset volume','Num_of_trades','Base_vol','Quote_vol','Ignr']

def _add_column_ma(df, ma_period, prefix='ma_', col='Close'):
    column_ma = prefix + str(ma_period)
    df.loc[0, column_ma] = df.loc[0, col]
    for i in range(1, len(df)):
        df.loc[i, column_ma] = MA.ema(df.loc[i, col], df.loc[i - 1, column_ma], ma_period)

    column_diff = prefix+'diff_' + str(ma_period)
    df[column_diff] = MA.ema_delta(df[column_ma].shift(1), df[column_ma])

# pandas.DataFrame.query -- выбрать строку DataFrame

period = '1h'
btcusdt = pd.DataFrame(client.klines("BTCUSDT", period, limit=200), columns=columns)
# btcusdt.set_index("Open time")
btcusdt['Close'] = btcusdt['Close'].astype('float')
short_ma = 5
middle_ma = 13
long_ma = 34
k = 2 / (short_ma + 1)
# btcusdt['my_ma_'+str(short_ma)] = btcusdt['my_ma_'+str(short_ma)].rolling(1).add(1)
btcusdt.loc[0, 'my_ma_'+str(short_ma)] = btcusdt.loc[0, 'Close']
btcusdt.loc[0, 'my_ma_'+str(middle_ma)] = btcusdt.loc[0, 'Close']
startma = btcusdt.at[0, 'Close']

_add_column_ma(btcusdt, short_ma)
_add_column_ma(btcusdt, middle_ma)
_add_column_ma(btcusdt, long_ma)

for i in range(1, len(btcusdt)):
    btcusdt.loc[i, 'my_ma_'+str(short_ma)] = \
        btcusdt.loc[i, 'Close'] * k + btcusdt.loc[i-1, 'my_ma_'+str(short_ma)] * (1 - k)

# btcusdt['ma_diff_'+str(short_ma)] = btcusdt['my_ma_'+str(short_ma)].shift(1) / btcusdt['my_ma_'+str(short_ma)] - 1
# btcusdt['diff_ma_'+str(middle_ma)] = btcusdt['ma_'+str(middle_ma)].shift(1) / btcusdt['ma_'+str(middle_ma)] -1
# btcusdt['diff_ma_'+str(long_ma)] = btcusdt['ma_'+str(long_ma)].shift(1) / btcusdt['ma_'+str(long_ma)] -1


In [15]:
btcusdt[['Open time', 'ma_diff_5', 'ma_diff_13', 'ma_diff_34']].tail(3)

,Open time,ma_diff_5,ma_diff_13,ma_diff_34
197,1657980000000,-0.001616,0.000794,0.000391
198,1657983600000,-0.003212,0.001601,0.000738
199,1657987200000,-0.002899,0.001699,0.000827


,a,b,c
0,1,0.0,0.0
1,2,5.0,10.0
2,3,1.0,2.0
3,4,0.0,0.0
4,5,1.0,2.0
5,6,0.0,0.0
6,7,6.0,12.0
7,8,1.0,2.0
8,9,2.0,4.0
9,10,0.0,0.0
